In [ ]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install spacy[nb]
!python -m spacy download nb_core_news_sm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-16 03:00:22.330464: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-16 03:00:22.330594: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-16 03:00:22.330615: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries.

In [ ]:
# Load the Norwegian (Bokmål) language model in Spacy
nlp = spacy.load("nb_core_news_sm")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the CSV file
df = pd.read_excel('/content/drive/MyDrive/dataset.xlsx')

In [ ]:
# Define a function to preprocess text data by removing stop words and lemmatizing
def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return " ".join(tokens)

In [ ]:
# Preprocess the text data and add to a new "processed_text" column
df["processed_text"] = df["Feilbeskrivelse"].apply(preprocess)

In [ ]:
# Define a function to extract entities from text using Spacy NER
def extract_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append(ent.text)
    return entities

In [ ]:
# Extract entities from each row of text and add to a new "entities" column
df["entities"] = df["Feilbeskrivelse"].apply(extract_entities)

In [ ]:
# Fit the TfidfVectorizer transformer on the preprocessed text data
tfidf = TfidfVectorizer()
tfidf_data = tfidf.fit_transform(df["processed_text"])

In [ ]:
# Define a function to perform semantic search using cosine similarity
def semantic_search(query, data):
    # Transform the query data using the fitted TfidfVectorizer object
    query_vector = tfidf.transform([preprocess(query)])
    cosine_similarities = cosine_similarity(query_vector, data).flatten()
    related_docs_indices = cosine_similarities.argsort()[::-1]
    return [(index, cosine_similarities[index]) for index in related_docs_indices]


In [ ]:
# Test the semantic search function on a sample query
query = "Fortell meg om batterika?"
results = semantic_search(query, tfidf_data)
print("Top results for query: {}".format(query))
for result in results[:5]:
    print("Document index: {}, cosine similarity: {}".format(result[0], result[1]))
    print(df.iloc[result[0]]["Feilbeskrivelse"])
    print()

Top results for query: Fortell meg om batterika?
Document index: 89, cosine similarity: 0.5231659524291754
Melding om brannvarsel i batterikas

Document index: 44136, cosine similarity: 0.49092354069434013
Fører forteller at han fikk strøm n

Document index: 66891, cosine similarity: 0.45859402774431196
Vektere ringer inn og forteller at 

Document index: 390, cosine similarity: 0.37236401653930623
402 Bry-øst brannvarsel i batterika

Document index: 25139, cosine similarity: 0.0
Ingen lyd i HMI (hører ikke varsler

